In [ ]:
import numpy as np
import pandas as pd
from decimal import Decimal
from uniswap import *
from datetime import datetime,timezone
from data_downloading import *






In [61]:
#We let token0 = WETH, token1 = USDC
token0 = 'WETH'
token1 = 'USDC'
decimal_0 = 18
decimal_1 = 6
fee_tier = 0.0005
tickspacing = 10

## Fetch data  

In [ ]:
# subgraph api,url
API_KEY = "replace with your subgraph api key"
url = f"https://gateway.thegraph.com/api/{API_KEY}/subgraphs/id/5zvR82QoaXYFyDEKLZ9t6v9adgnptxYpKpSbxtgVENFV"
pool_id = "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640"

In [106]:
swap_attr =['logIndex','timestamp','amount0',
        'amount1','amountUSD', 'sqrtPriceX96', 'tick']
liq_attr = ['timestamp','logIndex','amount0','amount1',
            'amount','tickLower','tickUpper']

In [ ]:
# Raw data
df_swap =query_data('swaps',url=url,pool_id=pool_id,swap_attr)
df_swap.rename(columns={'datetime':'time'},inplace=True)
df_swap.to_csv(f'./data/{token0}H&{token1}_{int(fee_tier*1000)}bp_mainnet_swap.csv')

df_mint =query_data('mints',url=url,pool_id=pool_id,liq_attr)
df_mint.rename(columns={'datetime':'time'},inplace=True)
df_mint.to_csv(f'./data/{token0}H&{token1}_{int(fee_tier*1000)}bp_mainnet_mint.csv')

df_burn =query_data('burns',url=url,pool_id=pool_id,liq_attr)
for col in df_burn.columns:
    df_burn[col] = pd.to_numeric(df_burn[col].apply(Decimal))

df_burn.loc[:,['amount','amount0','amount1']]=(df_burn[['amount','amount0','amount1']]*-1).copy()
df_burn.rename(columns={'datetime':'time'},inplace=True)

df_burn.to_csv(f'./data/{token0}H&{token1}_{int(fee_tier*1000)}bp_mainnet_burn.csv')


## Transform to human readable
Note that in raw data, `sqrtPriceX96` corresponds to the square root price of USDC w.r.t WETH. So it is important to be mindful of the price and tick direction. 


### Swap

In [ ]:
def sqrt_2_price(s):
    s = int(float(Decimal(s)) )
    return (transform_price_2_readable((s/2**96)**2))**-1 #take inverse to get WETH price


df_swap.rename(columns={'amount0':token1,'amount1':token0},inplace=True)
df_swap['price']=df_swap['sqrtPriceX96'].apply(sqrt_2_price)

In [ ]:
df_swap = df_swap.drop(columns='sqrtPriceX96')[['time','timestamp','logIndex','tick','USDC','WETH','price'] ]

end ='2024-04-29 19:00:00'
end_ts = int(datetime.strptime(end, "%Y-%m-%d %H:%M:%S").replace(tzinfo=timezone.utc).timestamp())
df_swap=df_swap[df_swap['timestamp']<=end_ts]
df_swap=df_swap.sort_values(['timestamp','logIndex'])
df_swap.loc[:,'tick'] = (df_swap.loc[:,'tick']*-1).astype(int)



### Mint/Burn

In [ ]:
df_mint.rename(columns={'amount0':token1,'amount1':token0},inplace=True)
df_burn.rename(columns={'amount0':token1,'amount1':token0},inplace=True)
df_mint['time']= pd.to_datetime(df_mint['timestamp'],unit='s')
df_burn['time']= pd.to_datetime(df_burn['timestamp'],unit='s')


df_mint=df_mint[df_mint['timestamp']<=end_ts]
df_burn=df_burn[df_burn['timestamp']<=end_ts]

df_mint.loc[:,'logIndex'] = df_mint['logIndex'].astype(int).copy()
df_burn.loc[:,'logIndex'] = df_burn['logIndex'].astype(int).copy()

df_mint =df_mint[[ 'time','timestamp','logIndex','amount', f'{token0}', f'{token1}',  'tickLower', 'tickUpper']]
df_burn =df_burn[[ 'time','timestamp','logIndex','amount', f'{token0}', f'{token1}',  'tickLower', 'tickUpper']]

df_mb = pd.concat([df_mint,df_burn]).sort_values(['timestamp','logIndex'])
df_mb=df_mb.reset_index().drop(columns='index')
df_mb.loc[:,['tickUpper','tickLower'] ]= (df_mb[['tickUpper','tickLower'] ]*-1).astype(int).copy()
df_mb.rename(columns={ 'tickUpper':'tickLower','tickLower':'tickUpper'},inplace=True)


## Liquidity profile

Compute the historical liquidity profile, consider the range from the historical minimum to the maximum tick, extended by an additional 770 ticks on both sides (≈$8\%$ in price)

In [ ]:
liq_profile(df_mb,df_swap['tick'].min()-770, df_swap['tick'].max()+770,tick_spacing=10)


## Add the liquidity before swap
Adding the liquidity before each trade to swap data.


In [ ]:

#read separatly and concat cost less memory than directly read the whole file
df_liq = pd.concat( [pd.read_parquet(f'./data/df_liq_{i}.parquet') for i in range(1,22)]).reset_index().drop(columns='index')
get_liq(df_liq,df_swap)
df_swap.to_csv(f'./data/{token0}&{token1}_5bp_mainnet_swap_with_liq.csv')



